# Chord2Vec

Based on https://www.tensorflow.org/text/tutorials/word2vec

In [1]:
import pickle
import os
import numpy as np
from itertools import chain
import io
import re
import string
import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [54]:


# navigate up one folder then down into the scraped folder
os.chdir('..')
os.chdir('scraped')

#use pickle to read chords_songs_0-1000.pkl
with open("chords_songs_0-999.pkl", 'rb') as f:
    upto_1000 = pickle.load(f)

#use pickle to read chords_songs_1000-1999.pkl
with open("chords_songs_1000-1999.pkl", 'rb') as f:
    upto_2000 = pickle.load(f)

#use pickle to read chords_songs_2000-2999.pkl
with open("chords_songs_2000-2999.pkl", 'rb') as f:
    upto_3000 = pickle.load(f)

#use pickle to read chords_songs_3000-4955.pkl
with open("chords_songs_3000-4955.pkl", 'rb') as f:
    upto_4955 = pickle.load(f)

# upto_1000 is a list of LISTS. Flatten it.
    
upto_3000

[['Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Cmaj7',
  'Bm',
  'Cmaj7',
  'Bm',
  'C',
  'Bm',
  'D',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Cmaj7',
  'Bm',
  'Cmaj7',
  'Bm',
  'C',
  'Bm',
  'D',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Em',
  'Em7',
  'A',
  'Cmaj7',
  'Bm',
  'Cmaj7',
  'Bm',
  'C',
  'Bm',
  'D',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A',
  'G',
  'D',
  'A'],
 ['C',
  'G/B',
  'Am',
  'C',

### 2 phases of flattening. 
first, flatten each batch of ~1000 songs 
then, flatten the 4 batches into one


In [55]:
flattened = []
for batch in [upto_1000, upto_2000, upto_3000, upto_4955]:
    for song in batch:
        for chord in song:
            flattened.append(chord)

len(flattened)

453122

### create numpy array of just the unique chords

In [56]:
allchords_arr = np.array(flattened)
unique_chords = np.unique(allchords_arr)
unique_chords.shape

(1115,)

### create a vocabulary dictionary holding just the unique chords labelled with numbers

In [57]:
# create a vocabulary dictionary holding just the unique chords labelled with numbers
vocab = {chord: i for i, chord in enumerate(unique_chords)}

# swap the keys and values of the inv_vocab dictionary
inv_vocab = {v: k for k, v in vocab.items()}
inv_vocab

{0: 'A',
 1: 'A#',
 2: 'A#/C#',
 3: 'A#2',
 4: 'A#5',
 5: 'A#7',
 6: 'A#dim',
 7: 'A#dim7',
 8: 'A#m',
 9: 'A#m/C#',
 10: 'A#m7',
 11: 'A#m7b5',
 12: 'A#m9',
 13: 'A#maj7',
 14: 'A#sus4',
 15: 'A(5)',
 16: 'A(7)',
 17: 'A(9)',
 18: 'A(b9)',
 19: 'A(maj9)',
 20: 'A+',
 21: 'A/B',
 22: 'A/Bb',
 23: 'A/C#',
 24: 'A/D',
 25: 'A/E',
 26: 'A/F',
 27: 'A/F#',
 28: 'A/G',
 29: 'A/G#',
 30: 'A11',
 31: 'A13',
 32: 'A13b5',
 33: 'A2',
 34: 'A4',
 35: 'A5',
 36: 'A6',
 37: 'A6/B',
 38: 'A6/C#',
 39: 'A6/E',
 40: 'A7',
 41: 'A7#5',
 42: 'A7(b9)',
 43: 'A7-9/E',
 44: 'A7/C#',
 45: 'A7/D',
 46: 'A7/E',
 47: 'A7/G',
 48: 'A7add11',
 49: 'A7add13',
 50: 'A7add6',
 51: 'A7b13',
 52: 'A7b5',
 53: 'A7b9',
 54: 'A7sus',
 55: 'A7sus2',
 56: 'A7sus2/E',
 57: 'A7sus4',
 58: 'A7sus4/E',
 59: 'A9',
 60: 'A9/C#',
 61: 'A9sus4',
 62: 'AM',
 63: 'AM7',
 64: 'Aadd#11',
 65: 'Aadd11',
 66: 'Aadd2/B',
 67: 'Aadd4',
 68: 'Aadd4/C#',
 69: 'Aadd9',
 70: 'Aadd9/B',
 71: 'Aadd9/C#',
 72: 'Aadd9/E',
 73: 'Aadd9/F#',
 74: 

### Convert one song into 'sequence' of numnbers mapping to chords in vocab



In [58]:
onesong = upto_1000[0]
onesong
example_sequence = [vocab[chord] for chord in onesong]
print(example_sequence)

[755, 950, 475, 0, 755, 950, 475, 0, 755, 950, 475, 0, 755, 950, 475, 0, 755, 755, 950, 475, 0, 755, 950, 475, 0, 755, 950, 475, 0, 755, 755, 950, 475, 0, 755, 755, 950, 475, 0, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 197, 755, 950, 475, 0, 755, 950, 475, 0, 755, 950, 475, 0, 755, 950, 475, 0, 755, 755, 950, 475, 0, 755, 950, 475, 0, 755, 950, 475, 0, 755, 755, 950, 475, 0, 755, 755, 950, 475, 0, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 197, 755, 950, 475, 0, 755, 950, 475, 0, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 169, 197, 755, 950, 475, 0, 755, 950, 475, 0, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 475, 755, 313, 950, 197, 755, 313, 475, 23, 950, 479, 755, 313, 475, 23, 950, 479, 755, 313, 475, 23, 950, 479, 755, 313, 475, 23, 950, 479]


### Negative sampling for one skip-gram

### Import pacakges

In [61]:
vocab_size = len(vocab)
window_size = 2

window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

534


In [62]:
positive_skip_grams[:5]

[[23, 313], [950, 313], [313, 755], [755, 475], [479, 755]]

### Print a few positive skip-grams
### (target_word, context_word)


In [63]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inv_vocab[target]}, {inv_vocab[context]})")

(23, 313): (A/C#, C)
(950, 313): (G, C)
(313, 755): (C, Em)
(755, 475): (Em, D)
(479, 755): (D#5, Em)


In [64]:
# Get target and context words for one positive skip-gram.
target_chord, context_chord = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

#context_class is a vector holding ONE context chord (which is close to the target chord) 
# then negative sampling candidates for the context chord are generated
# negative sampling candidaes are the words that are NOT context chords, thefore NOT likely to be close to the target chord

context_class = tf.reshape(tf.constant(context_chord, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=42,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inv_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([ 75  11 288  29], shape=(4,), dtype=int64)
['Aaug/Eb', 'A#m7b5', 'Bm6', 'A/G#']


### Construct one training example
(text taken from https://www.tensorflow.org/text/tutorials/word2vec) 


For a given positive (target_word, context_word) skip-gram, you now also have num_ns negative sampled context words that do not appear in the window size neighborhood of target_word. 

Batch the 1 positive context_word and num_ns negative context words into one tensor. This produces a set of positive skip-grams (labeled as 1) and negative samples (labeled as 0) for each target word.

In [65]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_chord

context

In [77]:
context

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([313,  75,  11, 288,  29], dtype=int64)>

In [73]:
target

23

Check out the context and the corresponding labels for the target word from the skip-gram example above:

context[0] = positive example
context[1:-1] = negative example

In [76]:
print(f"target_index    : {target}")
print(f"target_chord     : {inv_vocab[target_chord]}")
print(f"context_indices : {context}")
print(f"context_chords   : {[inv_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 23
target_chord     : A/C#
context_indices : [313  75  11 288  29]
context_chords   : ['C', 'Aaug/Eb', 'A#m7b5', 'Bm6', 'A/G#']
label           : [1 0 0 0 0]


# generate training data
all steps above compiled into one func.

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## Encode chord progressions as sequences
What are sequences?
like sentences represented as integers

In [3]:
vectorize_layer = layers.TextVectorization(
    standardize=None,
    max_tokens=vocab_size,
    output_mode='int')

NameError: name 'vocab_size' is not defined